In [1]:
# Important libraries for complete program execution

import json
import pandas as pd
import numpy as np

## Data Wrangling

The Dataset we have is in unstructured format (.txt file). Therefore, we first need to structure it and convert it into a format which is easier to work on.

In [2]:
with open("Datasets/Digital Nomad Travel Logs.txt") as file:
    data = file.readlines()

# Showing a sample of current data    
data[0][:1000]

'{"README":["Please mention and link back to Nomad List when you use this data","https:\\/\\/nomadlist.com\\/graph","Last updated 22 hours ago"],"lisbon-portugal":{"porto-portugal":134,"madrid-spain":66,"london-united-kingdom":57,"barcelona-spain":53,"paris-france":43,"amsterdam-netherlands":42,"berlin-germany":36,"lagos-algarve-portugal":24,"madeira-portugal":24,"ericeira-portugal":23},"madeira-portugal":{"lisbon-portugal":26,"berlin-germany":4,"las-palmas-canary-islands-spain":3,"porto-portugal":3,"london-united-kingdom":2,"barcelona-spain":2,"paris-france":2,"warsaw-poland":1,"corralejo-canary-islands-spain":1,"rio-de-janeiro-brazil":1},"porto-portugal":{"lisbon-portugal":105,"london-united-kingdom":34,"barcelona-spain":25,"madrid-spain":15,"amsterdam-netherlands":12,"berlin-germany":11,"paris-france":10,"krakow-poland":6,"madeira-portugal":6,"new-york-city-ny-united-states":6},"las-palmas-canary-islands-spain":{"madrid-spain":17,"london-united-kingdom":16,"tenerife-canary-islands-s

## String Dictionary to Dictionary

As we can see the data currently is a list with only 1 element which is a string dictionary. Our task for now is to convert that string dictionary into a dictionary. For this we will use the json package of Python

In [3]:
data_dict = json.loads(data[0])

# Showing dictionary sample in key value pair format
list(data_dict.items())[:2]

[('README',
  ['Please mention and link back to Nomad List when you use this data',
   'https://nomadlist.com/graph',
   'Last updated 22 hours ago']),
 ('lisbon-portugal',
  {'porto-portugal': 134,
   'madrid-spain': 66,
   'london-united-kingdom': 57,
   'barcelona-spain': 53,
   'paris-france': 43,
   'amsterdam-netherlands': 42,
   'berlin-germany': 36,
   'lagos-algarve-portugal': 24,
   'madeira-portugal': 24,
   'ericeira-portugal': 23})]

In [4]:
# Deleting 'README' key from the dictionary

del data_dict['README']

# Showing dictionary sample in key value pair format
list(data_dict.items())[:2]

[('lisbon-portugal',
  {'porto-portugal': 134,
   'madrid-spain': 66,
   'london-united-kingdom': 57,
   'barcelona-spain': 53,
   'paris-france': 43,
   'amsterdam-netherlands': 42,
   'berlin-germany': 36,
   'lagos-algarve-portugal': 24,
   'madeira-portugal': 24,
   'ericeira-portugal': 23}),
 ('madeira-portugal',
  {'lisbon-portugal': 26,
   'berlin-germany': 4,
   'las-palmas-canary-islands-spain': 3,
   'porto-portugal': 3,
   'london-united-kingdom': 2,
   'barcelona-spain': 2,
   'paris-france': 2,
   'warsaw-poland': 1,
   'corralejo-canary-islands-spain': 1,
   'rio-de-janeiro-brazil': 1})]

## Structuring the data

Now, we have a dictionary in which key of every element is the place from where the digital nomads travelled and the value contains a dictionary which shows the cities they travelled to and the number of times they travelled.
Now we have to convert this data into 3 python lists with each having values, the city from where the digital nomads travelled, the city where they travelled to, and the number of times they travelled respectively

In [5]:
from_cities = []
to_cities = []
number_of_trips = []

When we iterate over the dictonary, we can see that the format of dictionary is something like this:
The key of the key value pair represents the city from which the travel was made. The value on the other hand is again a dictionary where key represents the city to which the travel was made and value represents the number of times the travel was made.

In [6]:
# iterating over the dictionary to separate all its key value pairs and form 3 lists with required information

for key,value in data_dict.items():
    if type(value) == list:
        continue
    for k,v in value.items():
        from_cities.append(key)
        to_cities.append(k)
        number_of_trips.append(v)

In [7]:
df = pd.DataFrame(list(zip(from_cities, from_cities, to_cities, to_cities, number_of_trips)), 
                  columns = ['From City', 'From Country', 'To City', 'To Country', 'Number of Trips'])
df.head()

,From City,From Country,To City,To Country,Number of Trips
0,lisbon-portugal,lisbon-portugal,porto-portugal,porto-portugal,134
1,lisbon-portugal,lisbon-portugal,madrid-spain,madrid-spain,66
2,lisbon-portugal,lisbon-portugal,london-united-kingdom,london-united-kingdom,57
3,lisbon-portugal,lisbon-portugal,barcelona-spain,barcelona-spain,53
4,lisbon-portugal,lisbon-portugal,paris-france,paris-france,43


## Separating Country and City in 'From City' and 'From Country' Column

In [8]:
cities = []
countries = []

# checklist of countries which require cutting 3 words from the end
checklist_3words = ['united-arab-emirates', 'papua-new-guinea']

# checklist of countries which require cutting 2 words from the end
checklist_2words = ['new-zealand', 'south-korea', 'cote-divoire', 'cook-islands', 'south-africa', 'saudi-arabia', 
                    'sierra-leone', 'hong-kong', 'solomon-islands', 'dr-congo', 'united-kingdom', 'new-caledonia']

for city in df['From City']:
    country = ''

    # country and city both are of one word only
    if city.count('-') == 1:
        country = city.split('-')[1]
        city = city.split('-')[0]

    # country is United States
    elif 'united-states' in city:
        country = city.split('-')[-2:]
        country = ' '.join(country)
        city = city.split('-')[:-3]
        city = ' '.join(city)
    
    # country is of 3 words or requires cutting 3 words from the end
    elif any(c in city for c in checklist_3words):
        country = city.split('-')[-3:]
        country = ' '.join(country)
        city = city.split('-')[:-3]
        city = ' '.join(city)
    
    # country is of 2 words or requires cutting 2 words from the end
    elif any(c in city for c in checklist_2words):
        country = city.split('-')[-2:]
        country = ' '.join(country)
        city = city.split('-')[:-2]
        city = ' '.join(city)
        
    # country is of 1 word but city is more than 1 word long
    else:
        country = city.split('-')[-1]
        city = city.split('-')[:-1]
        city = ' '.join(city)
    
    cities.append(city)
    countries.append(country)

df['From City'] = cities
df['From Country'] = countries
df.head()

,From City,From Country,To City,To Country,Number of Trips
0,lisbon,portugal,porto-portugal,porto-portugal,134
1,lisbon,portugal,madrid-spain,madrid-spain,66
2,lisbon,portugal,london-united-kingdom,london-united-kingdom,57
3,lisbon,portugal,barcelona-spain,barcelona-spain,53
4,lisbon,portugal,paris-france,paris-france,43


## Separating Country and City in 'To City' and 'To Country' Column

In [9]:
cities = []
countries = []

# checklist of countries which require cutting 3 words from the end
checklist_3words = ['united-arab-emirates', 'papua-new-guinea']

# checklist of countries which require cutting 2 words from the end
checklist_2words = ['new-zealand', 'south-korea', 'cote-divoire', 'cook-islands', 'south-africa', 'saudi-arabia', 
                    'sierra-leone', 'hong-kong', 'solomon-islands', 'dr-congo', 'united-kingdom', 'new-caledonia']

for city in df['To City']:
    country = ''

    # country and city both are of one word only
    if city.count('-') == 1:
        country = city.split('-')[1]
        city = city.split('-')[0]

    # country is United States
    elif 'united-states' in city:
        country = city.split('-')[-2:]
        country = ' '.join(country)
        city = city.split('-')[:-3]
        city = ' '.join(city)
    
    # country is of 3 words or requires cutting 3 words from the end
    elif any(c in city for c in checklist_3words):
        country = city.split('-')[-3:]
        country = ' '.join(country)
        city = city.split('-')[:-3]
        city = ' '.join(city)
    
    # country is of 2 words or requires cutting 2 words from the end
    elif any(c in city for c in checklist_2words):
        country = city.split('-')[-2:]
        country = ' '.join(country)
        city = city.split('-')[:-2]
        city = ' '.join(city)
        
    # country is of 1 word but city is more than 1 word long
    else:
        country = city.split('-')[-1]
        city = city.split('-')[:-1]
        city = ' '.join(city)
    
    cities.append(city)
    countries.append(country)

df['To City'] = cities
df['To Country'] = countries
df.head()

,From City,From Country,To City,To Country,Number of Trips
0,lisbon,portugal,porto,portugal,134
1,lisbon,portugal,madrid,spain,66
2,lisbon,portugal,london,united kingdom,57
3,lisbon,portugal,barcelona,spain,53
4,lisbon,portugal,paris,france,43


## Capitalizing city and country names

In [10]:
df['From City'] = df['From City'].str.title()
df['From Country'] = df['From Country'].str.title()
df['To City'] = df['To City'].str.title()
df['To Country'] = df['To Country'].str.title()
df.head()

,From City,From Country,To City,To Country,Number of Trips
0,Lisbon,Portugal,Porto,Portugal,134
1,Lisbon,Portugal,Madrid,Spain,66
2,Lisbon,Portugal,London,United Kingdom,57
3,Lisbon,Portugal,Barcelona,Spain,53
4,Lisbon,Portugal,Paris,France,43


## Saving resultant dataframe as a CSV file

In [11]:
df.to_csv('Datasets/Digital Nomad Travel Logs.csv')